<center> 
<h1>ANALYTICS CASE STUDY<h1/>
<center/>

In [631]:
#importing neccesary libraries
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

In [632]:
#importing the case study files
gameplay_data = pd.read_csv('Gameplay data.csv', skiprows=3)
deposit_data = pd.read_csv('Deposit Data.csv', skiprows=3)
withdrawal_data = pd.read_csv('Withdrawal Data.csv', skiprows=3)

In [633]:
gameplay_data.head()

,User ID,Games Played,Datetime
0,851,1,01-10-2022 0:00
1,717,1,01-10-2022 0:00
2,456,1,01-10-2022 0:00
3,424,1,01-10-2022 0:00
4,845,1,01-10-2022 0:00


In [634]:
deposit_data.head()

,User Id,Datetime,Amount
0,357,01-10-2022 0:03,2000
1,776,01-10-2022 0:03,2500
2,492,01-10-2022 0:06,5000
3,803,01-10-2022 0:07,5000
4,875,01-10-2022 0:09,1500


In [635]:
withdrawal_data.head()

,User Id,Datetime,Amount
0,190,01-10-2022 0:03,5872
1,159,01-10-2022 0:16,9540
2,164,01-10-2022 0:24,815
3,946,01-10-2022 0:29,23000
4,763,01-10-2022 0:40,9473


**checking for null values**

In [636]:
gameplay_data.isna().sum()

User ID         0
Games Played    0
Datetime        0
dtype: int64

In [637]:
deposit_data.isna().sum()

User Id     0
Datetime    0
Amount      0
dtype: int64

In [638]:
withdrawal_data.isna().sum()

User Id     0
Datetime    0
Amount      0
dtype: int64

**checking for data types**

In [639]:
gameplay_data.dtypes

User ID          int64
Games Played     int64
Datetime        object
dtype: object

In [640]:
deposit_data.dtypes

User Id      int64
Datetime    object
Amount       int64
dtype: object

In [641]:
withdrawal_data.dtypes

User Id      int64
Datetime    object
Amount       int64
dtype: object

In [642]:
#converting date and time column from object type into date and time type
gameplay_data['Datetime'] = pd.to_datetime(gameplay_data['Datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'], format='%d-%m-%Y %H:%M', errors='coerce')

In [643]:
print('Gameplay_data-\n',gameplay_data.dtypes)
print()
print('Deposit_data-\n',deposit_data.dtypes)
print()
print('Withdrawal_data-\n',withdrawal_data.dtypes)

Gameplay_data-
 User ID                  int64
Games Played             int64
Datetime        datetime64[ns]
dtype: object

Deposit_data-
 User Id              int64
Datetime    datetime64[ns]
Amount               int64
dtype: object

Withdrawal_data-
 User Id              int64
Datetime    datetime64[ns]
Amount               int64
dtype: object


In [644]:
#Checking for duplicate values
print('Gameplay_data-\n',gameplay_data.duplicated().sum())
print()
print('Deposit_data-\n',deposit_data.duplicated().sum())
print()
print('Withdrawal_data-\n',withdrawal_data.duplicated().sum())
print()

Gameplay_data-
 13736

Deposit_data-
 1

Withdrawal_data-
 0



In [645]:
#checking number of rows and columns
print('Gameplay_data-\n',gameplay_data.shape)
print()
print('Deposit_data-\n',deposit_data.shape)
print()
print('Withdrawal_data-\n',withdrawal_data.shape)

Gameplay_data-
 (355266, 3)

Deposit_data-
 (17438, 3)

Withdrawal_data-
 (3566, 3)


In [646]:
#since there are duplicate values are present, we will drop the duplicate values 
gameplay_data.drop_duplicates(inplace = True)
deposit_data.drop_duplicates(inplace = True)

In [647]:
print('gameplay_data\n',gameplay_data.duplicated().sum())
print()
print('deposit_data\n',deposit_data.duplicated().sum())
print()
print('withdrawal_data\n',withdrawal_data.duplicated().sum())
print()

gameplay_data
 0

deposit_data
 0

withdrawal_data
 0



## 1. Find Playerwise Loyalty points earned by Players in the following slots:-

**Final Loyalty Point Formula
Loyalty Point = (0.01 * deposit) + (0.005 * Withdrawal amount) + (0.001 * (maximum of (#deposit - #withdrawal) or 0)) + (0.2 * Number of games played)**


**On each day, there are 2 slots for each of which the loyalty points are to be calculated:
S1 from 12am to 12pm 
S2 from 12pm to 12am**

**a. 2nd October Slot S1**

In [648]:
gp_2nd_oct = gameplay_data[(gameplay_data['Datetime'].dt.day == 2) & (gameplay_data['Datetime'].dt.month == 10) & (gameplay_data['Datetime'].dt.hour < 12)]

In [649]:
dd_2nd_oct = deposit_data[(deposit_data['Datetime'].dt.day == 2) & (deposit_data['Datetime'].dt.month == 10) & (deposit_data['Datetime'].dt.hour < 12)]

In [650]:
wd_2nd_oct = withdrawal_data[(withdrawal_data['Datetime'].dt.day == 2) & (withdrawal_data['Datetime'].dt.month == 10) & (withdrawal_data['Datetime'].dt.hour < 12)]

In [651]:
dd_2nd_oct.rename(columns={'Amount' : 'Deposit_amount','User Id':'User ID'}, inplace=True)

In [652]:
dd_2nd_oct.head()

,User ID,Datetime,Deposit_amount
607,695,2022-10-02 00:04:00,500
608,214,2022-10-02 00:07:00,2750
609,945,2022-10-02 00:07:00,5000
610,779,2022-10-02 00:08:00,5100
611,390,2022-10-02 00:09:00,2000


In [653]:
wd_2nd_oct.rename(columns={'Amount': 'Withdrawal_amount', 'User Id':'User ID'}, inplace=True)

In [654]:
wd_2nd_oct.head()

,User ID,Datetime,Withdrawal_amount
122,222,2022-10-02 00:23:00,60632
123,351,2022-10-02 00:30:00,5479
124,99,2022-10-02 00:36:00,43478
125,920,2022-10-02 00:46:00,13000
126,637,2022-10-02 00:51:00,3800


In [655]:
october2 = pd.concat([gp_2nd_oct, dd_2nd_oct,wd_2nd_oct], ignore_index = True)
oct_2nd = october2.groupby('User ID')[['Deposit_amount','Withdrawal_amount','Games Played']].sum().reset_index()

In [656]:
oct_2nd.shape

(636, 4)

In [657]:
oct_2nd

,User ID,Deposit_amount,Withdrawal_amount,Games Played
0,2,0.00,0.00,2.00
1,5,600.00,0.00,8.00
2,6,"4,000.00",0.00,2.00
3,7,0.00,0.00,5.00
4,8,0.00,0.00,6.00
...,...,...,...,...
631,990,0.00,0.00,67.00
632,992,0.00,0.00,40.00
633,996,0.00,0.00,11.00
634,997,0.00,0.00,5.00


**Final Loyalty Point Formula Loyalty Point = (0.01 * deposit) + (0.005 * Withdrawal amount) + (0.001 * (maximum of (#deposit - #withdrawal) or 0)) + (0.2 * Number of games played)**

In [658]:
def calculate_point(data):
    data['Loyalty Point'] = (
                            (0.01* data['Deposit_amount']) +
                            (0.005*data['Withdrawal_amount'])+
                            (0.001*(data['Deposit_amount']-data['Withdrawal_amount']).clip(lower=0))+
                            (0.2* data['Games Played'])
                            )
    return data
                                                              

In [659]:
october_2 = calculate_point(oct_2nd)
october_2

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point
0,2,0.00,0.00,2.00,0.40
1,5,600.00,0.00,8.00,8.20
2,6,"4,000.00",0.00,2.00,44.40
3,7,0.00,0.00,5.00,1.00
4,8,0.00,0.00,6.00,1.20
...,...,...,...,...,...
631,990,0.00,0.00,67.00,13.40
632,992,0.00,0.00,40.00,8.00
633,996,0.00,0.00,11.00,2.20
634,997,0.00,0.00,5.00,1.00


In [608]:
#Saving to local
october_2.to_excel('october_2.xlsx', index = False)

**b. 16th October Slot S2**

In [660]:
gp_16th_oct = gameplay_data[(gameplay_data['Datetime'].dt.day == 16) & (gameplay_data['Datetime'].dt.month == 10) & (gameplay_data['Datetime'].dt.hour >= 12)]
dd_16th_oct = deposit_data[(deposit_data['Datetime'].dt.day == 16) & (deposit_data['Datetime'].dt.month == 10) & (deposit_data['Datetime'].dt.hour >= 12)]
wd_16th_oct = withdrawal_data[(withdrawal_data['Datetime'].dt.day == 16) & (withdrawal_data['Datetime'].dt.month == 10) & (withdrawal_data['Datetime'].dt.hour >= 12)]

In [661]:
dd_16th_oct.rename(columns={'Amount' : 'Deposit_amount','User Id':'User ID'}, inplace=True)

In [662]:
dd_16th_oct.head()

,User ID,Datetime,Deposit_amount
8889,342,2022-10-16 12:04:00,2000
8890,114,2022-10-16 12:07:00,5000
8891,565,2022-10-16 12:10:00,50
8892,733,2022-10-16 12:13:00,1100
8893,695,2022-10-16 12:15:00,500


In [663]:
wd_16th_oct.rename(columns={'Amount': 'Withdrawal_amount', 'User Id':'User ID'}, inplace=True)

In [664]:
wd_16th_oct.head()

,User ID,Datetime,Withdrawal_amount
1788,82,2022-10-16 12:24:00,11610
1789,884,2022-10-16 12:30:00,20000
1790,125,2022-10-16 12:30:00,100
1791,497,2022-10-16 12:33:00,20000
1792,16,2022-10-16 12:36:00,6595


In [665]:
october16 = pd.concat([gp_16th_oct, dd_16th_oct,wd_16th_oct], ignore_index = True)
oct_16th = october16.groupby('User ID')[['Deposit_amount','Withdrawal_amount','Games Played']].sum().reset_index()

In [666]:
oct_16th

,User ID,Deposit_amount,Withdrawal_amount,Games Played
0,2,0.00,0.00,2.00
1,5,"1,000.00",0.00,9.00
2,6,0.00,0.00,1.00
3,8,0.00,0.00,5.00
4,9,"2,000.00",0.00,47.00
...,...,...,...,...
589,990,0.00,0.00,59.00
590,991,"6,000.00",0.00,1.00
591,992,0.00,0.00,41.00
592,996,0.00,0.00,9.00


In [667]:
october_16 = calculate_point(oct_16th)
october_16

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point
0,2,0.00,0.00,2.00,0.40
1,5,"1,000.00",0.00,9.00,12.80
2,6,0.00,0.00,1.00,0.20
3,8,0.00,0.00,5.00,1.00
4,9,"2,000.00",0.00,47.00,31.40
...,...,...,...,...,...
589,990,0.00,0.00,59.00,11.80
590,991,"6,000.00",0.00,1.00,66.20
591,992,0.00,0.00,41.00,8.20
592,996,0.00,0.00,9.00,1.80


In [611]:
october_16.to_excel('october_16.xlsx', index = False)

**b. 18th October Slot S1**

In [668]:
gp_18th_oct = gameplay_data[(gameplay_data['Datetime'].dt.day == 18) & (gameplay_data['Datetime'].dt.month == 10) & (gameplay_data['Datetime'].dt.hour < 12)]
dd_18th_oct = deposit_data[(deposit_data['Datetime'].dt.day == 18) & (deposit_data['Datetime'].dt.month == 10) & (deposit_data['Datetime'].dt.hour < 12)]
wd_18th_oct = withdrawal_data[(withdrawal_data['Datetime'].dt.day == 18) & (withdrawal_data['Datetime'].dt.month == 10) & (withdrawal_data['Datetime'].dt.hour < 12)]

In [669]:
dd_18th_oct.rename(columns={'Amount' : 'Deposit_amount','User Id':'User ID'}, inplace=True)

In [670]:
dd_18th_oct.head()

,User ID,Datetime,Deposit_amount
9741,180,2022-10-18 00:02:00,2000
9742,204,2022-10-18 00:06:00,2000
9743,749,2022-10-18 00:15:00,4000
9744,613,2022-10-18 00:16:00,1000
9745,536,2022-10-18 00:17:00,100


In [671]:
wd_18th_oct.rename(columns={'Amount': 'Withdrawal_amount', 'User Id':'User ID'}, inplace=True)

In [672]:
wd_18th_oct.head()

,User ID,Datetime,Withdrawal_amount
1949,441,2022-10-18 00:19:00,10000
1950,634,2022-10-18 00:20:00,316251
1951,193,2022-10-18 00:20:00,57000
1952,16,2022-10-18 00:46:00,17949
1953,47,2022-10-18 00:49:00,1611


In [673]:
october18 = pd.concat([gp_18th_oct, dd_18th_oct,wd_18th_oct], ignore_index = True)
oct_18th = october18.groupby('User ID')[['Deposit_amount','Withdrawal_amount','Games Played']].sum().reset_index()

In [674]:
oct_18th

,User ID,Deposit_amount,Withdrawal_amount,Games Played
0,2,0.00,0.00,2.00
1,3,0.00,0.00,2.00
2,5,0.00,0.00,8.00
3,7,0.00,0.00,3.00
4,8,0.00,0.00,9.00
...,...,...,...,...
617,990,0.00,0.00,67.00
618,992,0.00,0.00,47.00
619,996,800.00,0.00,5.00
620,997,0.00,0.00,1.00


In [675]:
october_18 = calculate_point(oct_18th)
october_18

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point
0,2,0.00,0.00,2.00,0.40
1,3,0.00,0.00,2.00,0.40
2,5,0.00,0.00,8.00,1.60
3,7,0.00,0.00,3.00,0.60
4,8,0.00,0.00,9.00,1.80
...,...,...,...,...,...
617,990,0.00,0.00,67.00,13.40
618,992,0.00,0.00,47.00,9.40
619,996,800.00,0.00,5.00,9.80
620,997,0.00,0.00,1.00,0.20


In [613]:
october_18.to_excel('october_18.xlsx', index = False)

**b. 26th October Slot S2**

In [676]:
gp_26th_oct = gameplay_data[(gameplay_data['Datetime'].dt.day == 26) & (gameplay_data['Datetime'].dt.month == 10) & (gameplay_data['Datetime'].dt.hour >= 12)]
dd_26th_oct = deposit_data[(deposit_data['Datetime'].dt.day == 26) & (deposit_data['Datetime'].dt.month == 10) & (deposit_data['Datetime'].dt.hour >= 12)]
wd_26th_oct = withdrawal_data[(withdrawal_data['Datetime'].dt.day == 26) & (withdrawal_data['Datetime'].dt.month == 10) & (withdrawal_data['Datetime'].dt.hour >= 12)]

In [677]:
dd_26th_oct.rename(columns={'Amount' : 'Deposit_amount','User Id':'User ID'}, inplace=True)

In [678]:
dd_26th_oct.head()

,User ID,Datetime,Deposit_amount
14383,422,2022-10-26 12:01:00,15000
14384,516,2022-10-26 12:04:00,2000
14385,516,2022-10-26 12:06:00,5300
14386,875,2022-10-26 12:10:00,800
14387,553,2022-10-26 12:12:00,550


In [679]:
wd_26th_oct.rename(columns={'Amount': 'Withdrawal_amount', 'User Id':'User ID'}, inplace=True)

In [680]:
wd_26th_oct.head()

,User ID,Datetime,Withdrawal_amount
2911,542,2022-10-26 12:39:00,6000
2912,440,2022-10-26 12:50:00,1000
2913,9,2022-10-26 13:02:00,20076
2914,372,2022-10-26 13:08:00,20000
2915,880,2022-10-26 13:15:00,15000


In [681]:
october26 = pd.concat([gp_26th_oct, dd_26th_oct,wd_26th_oct], ignore_index = True)
oct_26th = october26.groupby('User ID')[['Deposit_amount','Withdrawal_amount','Games Played']].sum().reset_index()

In [682]:
oct_26th

,User ID,Deposit_amount,Withdrawal_amount,Games Played
0,2,"90,000.00",0.00,0.00
1,5,"1,000.00",0.00,4.00
2,6,0.00,0.00,1.00
3,7,0.00,"6,617.00",1.00
4,8,0.00,0.00,7.00
...,...,...,...,...
623,992,0.00,"30,000.00",41.00
624,995,0.00,0.00,1.00
625,996,0.00,0.00,4.00
626,997,0.00,0.00,2.00


In [683]:
october_26 = calculate_point(oct_26th)
october_26

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point
0,2,"90,000.00",0.00,0.00,990.00
1,5,"1,000.00",0.00,4.00,11.80
2,6,0.00,0.00,1.00,0.20
3,7,0.00,"6,617.00",1.00,33.29
4,8,0.00,0.00,7.00,1.40
...,...,...,...,...,...
623,992,0.00,"30,000.00",41.00,158.20
624,995,0.00,0.00,1.00,0.20
625,996,0.00,0.00,4.00,0.80
626,997,0.00,0.00,2.00,0.40


In [615]:
october_26.to_excel('october_26.xlsx', index = False)

## 2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October.  
## In case of tie, number of games played should be taken as the next criteria for ranking.

In [684]:
gameplay_data['Datetime'].dt.month.value_counts()

Datetime
10    341530
Name: count, dtype: int64

In [685]:
gp = gameplay_data.copy()

In [686]:
dd = deposit_data.copy()

In [687]:
wd = withdrawal_data.copy()

In [688]:
wd

,User Id,Datetime,Amount
0,190,2022-10-01 00:03:00,5872
1,159,2022-10-01 00:16:00,9540
2,164,2022-10-01 00:24:00,815
3,946,2022-10-01 00:29:00,23000
4,763,2022-10-01 00:40:00,9473
...,...,...,...
3561,559,2022-10-31 23:27:00,5000
3562,407,2022-10-31 23:51:00,3000
3563,389,2022-10-31 23:56:00,14481
3564,11,2022-10-31 23:57:00,4000


In [689]:
dd.rename(columns={'Amount' : 'Deposit_amount','User Id':'User ID'}, inplace=True)

In [690]:
dd.head()

,User ID,Datetime,Deposit_amount
0,357,2022-10-01 00:03:00,2000
1,776,2022-10-01 00:03:00,2500
2,492,2022-10-01 00:06:00,5000
3,803,2022-10-01 00:07:00,5000
4,875,2022-10-01 00:09:00,1500


In [691]:
wd.rename(columns={'Amount': 'Withdrawal_amount', 'User Id':'User ID'}, inplace=True)

In [692]:
wd.head()

,User ID,Datetime,Withdrawal_amount
0,190,2022-10-01 00:03:00,5872
1,159,2022-10-01 00:16:00,9540
2,164,2022-10-01 00:24:00,815
3,946,2022-10-01 00:29:00,23000
4,763,2022-10-01 00:40:00,9473


In [693]:
october_temp = pd.concat([gp, dd, wd], ignore_index = True)
october = october_temp.groupby('User ID')[['Deposit_amount','Withdrawal_amount','Games Played']].sum().reset_index()

In [694]:
# formatting float, rounding only to 2
pd.options.display.float_format = '{:,.2f}'.format

In [695]:
october

,User ID,Deposit_amount,Withdrawal_amount,Games Played
0,0,0.00,0.00,15.00
1,1,"5,000.00",0.00,8.00
2,2,"567,000.00","1,270,215.00",96.00
3,3,"40,000.00",0.00,80.00
4,4,"1,750.00",0.00,5.00
...,...,...,...,...
995,995,"34,500.00",0.00,18.00
996,996,"96,400.00",0.00,470.00
997,997,0.00,0.00,109.00
998,998,"7,400.00",0.00,4.00


In [696]:
overall_october = calculate_point(october)
overall_october

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point
0,0,0.00,0.00,15.00,3.00
1,1,"5,000.00",0.00,8.00,56.60
2,2,"567,000.00","1,270,215.00",96.00,"12,040.28"
3,3,"40,000.00",0.00,80.00,456.00
4,4,"1,750.00",0.00,5.00,20.25
...,...,...,...,...,...
995,995,"34,500.00",0.00,18.00,383.10
996,996,"96,400.00",0.00,470.00,"1,154.40"
997,997,0.00,0.00,109.00,21.80
998,998,"7,400.00",0.00,4.00,82.20


In [697]:
overall = overall_october.copy()

In [698]:
overall['Rank'] = overall.sort_values(by=['Loyalty Point', 'Games Played'], ascending=[False,False])['Loyalty Point'].rank(method='min', ascending=False)

In [699]:
overall['Rank'] = overall['Rank'].astype(int)

In [700]:
ranked = overall.sort_values(by='Rank')

In [701]:
ranked.to_excel('overall_october.xlsx', index = False)

**3. What is the average deposit amount?**


In [702]:
print('Average deposit amount:', deposit_data['Amount'].mean().round(2))

Average deposit amount: 5492.47


**4. What is the average deposit amount per user in a month?**

In [703]:
dep_per_user = deposit_data.groupby('User Id')['Amount'].sum()

In [704]:
print('Average deposit amount per user:',dep_per_user.mean().round(2))

Average deposit amount per user: 104669.1


**5. What is the average number of games played per user?**

In [705]:
gp_per_user = gameplay_data.groupby('User ID')['Games Played'].sum()

In [706]:
print('Average number of games played per user:',gp_per_user.mean())

Average number of games played per user: 341.531


**Part B - How much bonus should be allocated to leaderboard players?**

**After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard. The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.**

**Now the company needs to determine how much bonus money should be given to the players.**

**Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?**

**That’s for you to figure out.**

**Suggest a suitable way to divide the allocated money keeping in mind the following points:**
**1. Only top 50 ranked players are awarded bonus**

Answer : I would suggest it should be based on the loyalty points because it shows the overall contribution of the player in the game, if we consider only the total number of games played, it would be unfair for users who have deposited their hard earned money into the game, so finally its fair to say that they Should base it on the amount of loyalty points earned by the user

loaylty points clearly shows the engagement of the users in the game, so top ranked users should be awarded with more bonus

### Dividing the players into sub categories

#### Winner - rank 1
#### runner_ups - rank 2 to 5
#### Elite - rank 6 to 15
#### Legend - rank 16 to 30
#### Champion - rank 31 to 40
#### Challenger - rank 41 to 50

In [707]:
top_50 = ranked[:50]
top_50

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank
634,634,"515,000.00","15,737,705.00",24.00,"83,843.33",1
672,672,"2,158,700.00","233,750.00",10.00,"24,682.70",2
99,99,"1,164,800.00","2,403,141.00",10.00,"23,665.71",3
212,212,"1,924,981.00","589,850.00",1.00,"23,534.39",4
566,566,"1,819,175.00","185,071.00",182.00,"20,787.61",5
740,740,"1,738,490.00","365,288.00",2.00,"20,584.94",6
714,714,"1,676,300.00",0.00,6.00,"18,440.50",7
421,421,"878,600.00","1,269,809.00","1,532.00","15,441.44",8
30,30,"1,329,000.00","152,145.00",13.00,"15,230.18",9
222,222,"1,285,000.00","99,358.00",10.00,"14,534.43",10


In [708]:
Winner = ranked[:1]
runner_ups = ranked[1:5]
Elite = ranked[5:15]
Legend = ranked[15:30]
Champion = ranked[30:40]
Challenger = ranked[40:50]

In [709]:
Bonus_pool = 50000*0.1
Winner['Bonus Money'] = Bonus_pool

In [710]:
Bonus_pool = 50000*0.20
runner_ups['Bonus Money'] = Bonus_pool/len(runner_ups)

In [711]:
Bonus_pool = 50000*0.21
Elite['Bonus Money'] = Bonus_pool/len(Elite)

In [712]:
Bonus_pool = 50000*0.27
Legend['Bonus Money'] = Bonus_pool/len(Legend)

In [713]:
Bonus_pool = 50000*0.12
Champion['Bonus Money'] = Bonus_pool/len(Champion)

In [714]:
Bonus_pool = 50000*0.10
Challenger['Bonus Money'] = Bonus_pool/len(Challenger)

In [715]:
Winner

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
634,634,"515,000.00","15,737,705.00",24.00,"83,843.33",1,"5,000.00"


In [716]:
runner_ups.head(1)

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
672,672,"2,158,700.00","233,750.00",10.00,"24,682.70",2,"2,500.00"


In [717]:
Elite.head(1)

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
740,740,"1,738,490.00","365,288.00",2.00,"20,584.94",6,"1,050.00"


In [718]:
Legend.head(1)

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
920,920,"367,700.00","1,734,480.00",922.00,"12,533.80",16,900.00


In [719]:
Champion.head(1)

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
678,678,"379,000.00","1,100,000.00",12.00,"9,292.40",31,600.00


In [720]:
Challenger.head(1)

,User ID,Deposit_amount,Withdrawal_amount,Games Played,Loyalty Point,Rank,Bonus Money
515,515,"580,000.00","400,000.00",412.00,"8,062.40",41,500.00


**Part C**

**Would you say the loyalty point formula is fair or unfair?**

**Can you suggest any way to make the loyalty point formula more robust?**

answer: currrent loyalty point formula can be considered reasonably fair because it takes into account of multiple activity done by the user rather than considering only 1 metric

Assign more points for users who engage more frequently
for more games played,
also we can reward more points to frequent depositors , we can set a certain threshold after which more points will be multiplied with deposit amount

we can also introduce limit for accumulating excessive loyalty points for depositors and withdrawals

we can introduce daily login bonus or lucky try to win bonus or points

Use of public leaderboards so the players can track and compare progress globally and with friends

Invite bonus should be introduced for the person inviting as well as person joining.